## Accessing SQL via Python
In this continuation of the previous notebook, we show how to wrap the DB functions in a library to create a more scalable way to use our code. 

We start by reading in the file as before. Note the import of our `wedge_helper` library, which is stored in a file with that same name and the extension `py`. Open that file up (maybe in Spyder?) so that you can see what's inside.

In [1]:
import sqlite3
from wedge_helper import *

working_dir = "C:/Users/jchan/Dropbox/Teaching/2016_Fall/ADA/Topics/SQL/InClass/"
input_file = "OwnerTransactions_30.txt"

# Let's just open the file and read the first few lines to the screen.
with open(working_dir + input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split("\t"))
        if idx > 3 :
            break

['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id']
['2010-01-01 10:19:32', '6', '68', '2', '2.79DP9', 'GEN MERCH', 'D', ' ', ' ', '9', '2', '0', '0.0000', '2.7900', '5.5800', '2.7900', '0.0000', '1', '0', '0', '0', '0.0000', '0.0000', '3', '0', '0', '0.00000000', '2', '0', '0', '0.0000', '0', '0', '', '0', '0', '0', '0', '0', '0', 'NULL', '0', 'NULL', '', '0', '40825', '1', '0', '0', '1']
['2010-01-01 10:19:46', '6', '68', '2', '0000000

Let's do our standard open the DB in memory and create a cursor.

In [6]:
db = sqlite3.connect(working_dir + "change_me.db") #':memory:') # Make this a directory + file if you want to store the results.
cur = db.cursor()

init_db(cur) # take a look at the .py file to see what happened here.

In [7]:
with open(working_dir + input_file,'r') as ifile :
    populate_db(db,ifile,delimiter="\t",limit=None)

And now let's run a query and print out the results in a semi-pretty fashion. Check out [this page](https://docs.python.org/3.2/library/string.html#format-specification-mini-language) to learn more about the formatting tricks.

In [10]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')

for idx,row in enumerate(result) :
    print("On {1}, card_no = {0} spent {2:,.2f}.".format(row[0],row[1],row[2]))
    if idx > 20 :
        break

On 2010-01-03, card_no = 10179 spent 135.62.
On 2010-01-05, card_no = 10179 spent 25.82.
On 2010-01-06, card_no = 10179 spent 20.65.
On 2010-01-07, card_no = 10179 spent 4.79.
On 2010-01-09, card_no = 10179 spent 11.23.
On 2010-01-10, card_no = 10179 spent 155.29.
On 2010-01-13, card_no = 10179 spent 22.33.
On 2010-01-15, card_no = 10179 spent 40.66.
On 2010-01-17, card_no = 10179 spent 56.43.
On 2010-01-20, card_no = 10179 spent 40.68.
On 2010-01-22, card_no = 10179 spent 23.65.
On 2010-01-23, card_no = 10179 spent 5.29.
On 2010-01-24, card_no = 10179 spent 107.06.
On 2010-01-27, card_no = 10179 spent 13.76.
On 2010-02-03, card_no = 10179 spent 59.22.
On 2010-02-04, card_no = 10179 spent 71.62.
On 2010-02-09, card_no = 10179 spent 31.90.
On 2010-02-10, card_no = 10179 spent 38.94.
On 2010-02-14, card_no = 10179 spent 55.90.
On 2010-02-17, card_no = 10179 spent 32.17.
On 2010-02-18, card_no = 10179 spent 10.36.
On 2010-02-19, card_no = 10179 spent 8.03.


In [9]:
row

(10179, '2010-02-19', 8.03)

Now let's return to our big query from last class.

In [16]:
result = cur.execute('''SELECT card_no,
                                   department,
                                   substr(date(datetime),1,4) AS year,
                                   substr(date(datetime),6,2) AS month,
                                   sum(total) AS spend,
                                   count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
                                   sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                                   FROM transactions
                                   WHERE department != 0 and
                                        department != 15 and
                                        trans_status != 'M' and
                                        trans_status != 'C' and
                                        trans_status != 'J' and
                                       (trans_status = '' or 
                                        trans_status = ' ' or 
                                        trans_status = 'V' or 
                                        trans_status = 'R') and card_no = 18736
                          GROUP BY card_no, department, year, month
                          ORDER BY year, month''')

In [17]:
for row in result :
    print(row)

(18736, 1, '2011', '02', 55.690000000000005, 2, 12)
(18736, 2, '2011', '02', 30.08, 1, 14)
(18736, 3, '2011', '02', 5.68, 1, 1)
(18736, 4, '2011', '02', 9.68, 1, 2)
(18736, 5, '2011', '02', 10.32, 1, 2)
(18736, 8, '2011', '02', 16.13, 1, 2)
(18736, 9, '2011', '02', 0.15, 1, 1)
(18736, 13, '2011', '02', 33.879999999999995, 1, 5)
(18736, 18, '2011', '02', 7.99, 1, 1)
(18736, 1, '2011', '03', 115.89000000000003, 8, 23)
(18736, 2, '2011', '03', 132.49999999999997, 11, 57)
(18736, 3, '2011', '03', 11.31, 2, 3)
(18736, 4, '2011', '03', 43.519999999999996, 9, 13)
(18736, 5, '2011', '03', 18.3, 4, 4)
(18736, 8, '2011', '03', 7.1, 1, 1)
(18736, 12, '2011', '03', 0.93, 2, 3)
(18736, 13, '2011', '03', 117.73000000000002, 10, 21)
(18736, 18, '2011', '03', 31.96, 2, 4)
(18736, 1, '2011', '04', 144.41, 9, 31)
(18736, 2, '2011', '04', 125.30999999999999, 8, 44)
(18736, 3, '2011', '04', 29.57, 5, 6)
(18736, 4, '2011', '04', 28.76, 6, 9)
(18736, 5, '2011', '04', 26.6, 4, 6)
(18736, 6, '2011', '04', 14.

In [ ]:
for idx,row in enumerate(result) :
    print(row)
    if idx > 10 :
        break

Run the cell below as a best practice, but the DB will close if you quit the notebook. If it's an in-memory DB then it's also gone!

In [ ]:
db.close()